# Praktikum 3: Softmax, Regularisierung, Momentum und Batchsize
## 4. Eigener Datensatz
Suchen Sie sich eine verfügbare PyTorch MLP-Implementierung für einen Datensatz heraus, der Sie besonders interessiert. Reproduzieren Sie die Ergebnisse lokal, auf colab oder RosettaHub. Ist die Implementierung vergleichbar mit der hier gegebenen IMDB Implementierung? Wo unterscheidet sich diese und ggf. warum? Wie verändern Sich die Ergebnisse, wenn Sie bestimmte Hyperparameter verändern?

Quelle: https://github.com/christianversloot/machine-learning-articles/blob/main/creating-a-multilayer-perceptron-with-pytorch-and-lightning.md


In [5]:
import os
import torch
# from torch import nn
# # from torchvision.datasets import CIFAR10
# from torch.utils.data import DataLoader
# # from torchvision import transforms

OSError: [WinError 127] Die angegebene Prozedur wurde nicht gefunden. Error loading "c:\Users\anika\anaconda3\envs\GINN\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [ ]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Flatten(),
      nn.Linear(32 * 32 * 3, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 10)
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

In [ ]:
# Prepare CIFAR-10 dataset
dataset = CIFAR10(os.getcwd(), download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)

In [ ]:
# Initialize the MLP
mlp = MLP()

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [ ]:
 # Run the training loop
for epoch in range(0, 5): # 5 epochs at maximum

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
        
        # Get inputs
        inputs, targets = data
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform forward pass
        outputs = mlp(inputs)
        
        # Compute loss
        loss = loss_function(outputs, targets)
        
        # Perform backward pass
        loss.backward()
        
        # Perform optimization
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
        if i % 500 == 499:
            print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
            current_loss = 0.0

    # Process is complete.
print('Training process has finished.')